# Arduino Uno DAQ (Serial → Python)

This notebook is focused on **single channel data acquisition** (DAQ) from an Arduino connected over USB Serial. 

**Workflow:**
1. Overview
2. Hardware setup + wiring / channel map
3. Install + imports
4. DAQ Config
5. Acquire
6. Live view
7. Save data + metadata + Post plot



## 2. Hardware setup + wiring / channel map

**Arduino sketch expectations**
- The Arduino should stream lines like: `arduino_us,raw` then `1234567,512`
- `arduino_us` = `micros()` on the Arduino
- `raw` = ADC count (0–1023 on Uno) from one analog input
- A sample Arduino Sketch is provided here: [https://github.com/mrsoltys/GEEN3853-Data-Acquisition/blob/main/Arduino_Uno_to_Python_DAQ/Arduino_Uno_to_Python_DAQ.ino](https://github.com/mrsoltys/GEEN3853-Data-Acquisition/blob/main/Arduino_Uno_to_Python_DAQ/Arduino_Uno_to_Python_DAQ.ino)

**Channel map**
- `A0` → `ch0_V` (converted to volts in Python)

**Wiring reminder**
- Signal → A0
- Ground → GND
- Keep voltages within the Arduino ADC range (typically 0–5V unless you know what you're doing).


## 3. Install + imports

If you're using conda, install once:

```bash
conda install -c conda-forge ipympl pyserial pandas matplotlib numpy
```

Then restart the kernel.

In [ ]:
# Interactive plotting backend (recommended for live view)
%matplotlib widget

import time
from datetime import datetime
from collections import deque
import threading

import numpy as np
import pandas as pd

import serial
import serial.tools.list_ports

import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation

## 4. DAQ Config
# List available serial ports
Unplug/replug the Arduino and re-run this cell if you're unsure which port is your Arduino.

In [ ]:
ports = list(serial.tools.list_ports.comports())

if not ports:
    print("No serial ports found. Plug in the Arduino and try again.")
else:
    print("Available serial ports:")
    for p in ports:
        print(f"  {p.device}  —  {p.description}")

# Connect to the Arduino
Edit `PORT` to match your Arduino port from the list above.

- macOS often looks like: `/dev/cu.usbmodemXXXX`
- Windows often looks like: `COM5`

Also check that `BAUD` matches the baud rate specified in `Serial.begin()` in your Arduino sketch

Finally, set `sample_rate_hz` to  match your Arduino Sketch. Note: This python script does not control the sampling frequency of the Arduino, that is done in Arduino. Give an approprate `duration_s` for how long you'd like to aquire data.


In [ ]:
# --- Arduino / ADC config ---
PORT = '/dev/cu.usbmodem1101'   # Must match your Serial Port listed above (e.g., 'COM5' on Windows)
BAUD = 115200                   # must match Serial.begin(...) in your sketch
sample_rate_hz = 500.0          # target rate. Must match rate in Arduino Sketch
duration_s = 30.0               # How Long to gather data [seconds]
n_samples = int(round(sample_rate_hz * duration_s))
print(f'n_samples = {n_samples} (target)')

# --- Core DAQ config  ---
channels = ['ch0_V']
units = {'ch0_V': 'V'}
v_ref = 5.0                     # ADC reference voltage (V) 5V on Uno. 3.3V on Teensy4.1
adc_max = 2^10-1                # 10-bit ADC on Uno. 12 Bit on Teensy 

# --- Live View ----
live_view = True              # Set to True if you want a live plot as you record data
chunk_samples = 50            # UI update / buffer chunk size (not a hardware setting)
plot_window_s = 5.0           # seconds shown in the live view window

# Connect (opening the port typically resets the Arduino)
ser = serial.Serial(PORT, BAUD, timeout=1)
print(f"Connected to {PORT} at {BAUD} baud")

## 5. Acquire
This section acquires **exactly** `n_samples` and stores results in memory.

Live plot will show below if `live_view = True`

In [ ]:
# --- Acquire + Live View (single cell), using your proven thread + FuncAnimation approach ---

# Plot buffers: keep the most recent N points
# If you want this tied to plot_window_s and sample_rate_hz, you can replace N with:
# N = int(max(10, round(plot_window_s * sample_rate_hz)))
N = 500
tbuf = deque(maxlen=N)   # time (s)
vbuf = deque(maxlen=N)   # volts (V) computed from raw

# Log buffer: store the entire run for CSV export / df build
log_rows = []

stop_flag = False
t0_pc = time.time()

def parse_line(line: str):
    '''
    Expected line format from Arduino:
        us,raw   OR  arduino_us,raw
        1234567,512
    Returns (arduino_us:int, raw:int) or None.
    '''
    s = line.strip()
    if not s:
        return None
    sl = s.lower()
    # ignore header lines
    if sl.startswith("us") or sl.startswith("arduino_us"):
        return None

    parts = s.split(",")
    if len(parts) != 2:
        return None

    try:
        arduino_us = int(parts[0].strip())
        raw = int(parts[1].strip())
        return arduino_us, raw
    except ValueError:
        return None

def reader():
    '''
    Background thread: drains the serial port continuously.
    Stops when we reach n_samples OR stop_flag is set.
    Keep this loop lightweight to reduce the chance of falling behind.
    '''
    global stop_flag
    ser.reset_input_buffer()

    while not stop_flag:
        # Stop automatically once we've collected enough samples
        if len(log_rows) >= n_samples:
            stop_flag = True
            break

        line = ser.readline().decode("utf-8", errors="ignore")
        parsed = parse_line(line)
        if parsed is None:
            continue
            
        # THIS LINE GETS THE DATA!
        arduino_us, raw = parsed

        # Convert for plotting + storage using your existing calibration vars
        # (adc_max and v_ref should already exist in your notebook)
        t_s = arduino_us / 1_000_000.0
        volts = (raw / adc_max) * v_ref

        # Plot buffers (rolling window)
        tbuf.append(t_s)
        vbuf.append(volts)

        # Full log (for df/CSV)
        log_rows.append({
            "pc_time_s": time.time() - t0_pc,
            "arduino_us": arduino_us,
            "raw": raw,
            "ch0_V": volts,   # keep compatible with your df schema
        })

# --- If live_view is False: just acquire in the main thread (simple + reliable) ---
if not live_view:
    print("live_view is False — acquiring without live plot...")

    ser.reset_input_buffer()
    rows = []
    while len(rows) < n_samples:
        line = ser.readline().decode("utf-8", errors="ignore")
        parsed = parse_line(line)
        if parsed is None:
            continue
        arduino_us, raw = parsed
        volts = (raw / adc_max) * v_ref
        rows.append((arduino_us, volts))

    arduino_us_arr = np.array([r[0] for r in rows], dtype=np.int64)
    v_arr = np.array([r[1] for r in rows], dtype=np.float64)
    time_s = (arduino_us_arr - arduino_us_arr[0]) * 1e-6

    df = pd.DataFrame({
        "time_s": time_s,
        "ch0_V": v_arr,
        "arduino_us": arduino_us_arr,
    })

    print(df.head())
    print(f"Collected {len(df)} samples")

else:
    # --- Live view path (thread + FuncAnimation) ---
    thread = threading.Thread(target=reader, daemon=True)
    thread.start()
    print("Reader thread started.")

    fig, ax = plt.subplots()
    (line_plot,) = ax.plot([], [])
    ax.set_xlabel("Arduino time (s)")
    ax.set_ylabel("Volts (V)")
    ax.grid(True)
    ax.set_title("Live DAQ (Arduino → Python)")

    def init():
        line_plot.set_data([], [])
        return (line_plot,)

    def update(_frame):
        # Update plot
        if len(tbuf) >= 2:
            x = list(tbuf)
            y = list(vbuf)
            line_plot.set_data(x, y)
            ax.relim()
            ax.autoscale_view()

        # Stop condition: once we have enough samples, stop the reader + animation
        if len(log_rows) >= n_samples:
            global stop_flag
            stop_flag = True
            try:
                ani.event_source.stop()
            except Exception:
                pass

        return (line_plot,)

    ani = FuncAnimation(
        fig,
        update,
        init_func=init,
        interval=50,
        blit=False,
        cache_frame_data=False
    )

    plt.show()

    # Wait for thread to exit cleanly (with a timeout guard)
    thread.join(timeout=2.0)

    # Build df using your required columns
    # Convert Arduino time to start at 0 (seconds)
    arduino_us_arr = np.array([r["arduino_us"] for r in log_rows[:n_samples]], dtype=np.int64)
    v_arr = np.array([r["ch0_V"] for r in log_rows[:n_samples]], dtype=np.float64)
    time_s = (arduino_us_arr - arduino_us_arr[0]) * 1e-6

    df = pd.DataFrame({
        "time_s": time_s,
        "ch0_V": v_arr,
        "arduino_us": arduino_us_arr,
    })

    print(df.head())
    print(f"Collected {len(df)} samples with live view")

## 7. Save data + metadata + post plot
This section exports a CSV (Excel-friendly) and a small JSON metadata file.

In [ ]:
# Compute achieved / effective sample rate from time stamps
if len(df) > 1:
    dt = np.diff(df['time_s'].values)
    effective_sample_rate_hz = 1.0 / np.median(dt)
else:
    effective_sample_rate_hz = float('nan')

run_id = datetime.now().strftime('%Y%m%d_%H%M%S')
base = f'arduino_daq_{run_id}'
csv_path = base + '.csv'
json_path = base + '.json'

# Save CSV
df[['time_s', 'ch0_V']].to_csv(csv_path, index=False)
print('Saved CSV:', csv_path)

# Save metadata JSON
metadata = {
    'device_type': 'arduino_uno',
    'requested_sample_rate_hz': sample_rate_hz,
    'effective_sample_rate_hz': effective_sample_rate_hz,
    'duration_s': duration_s,
    'n_samples': int(len(df)),
    'channels': channels,
    'units': units,
    'baud': BAUD,
    'v_ref': v_ref,
    'adc_max': adc_max,
}

import json
with open(json_path, 'w') as f:
    json.dump(metadata, f, indent=2)
print('Saved metadata:', json_path)

# Post plot (static)
plt.figure()
plt.plot(df['time_s'], df['ch0_V'])
plt.xlabel('Time (s)')
plt.ylabel('Voltage (V)')
plt.title(f'Arduino Uno | {len(df)} samples | effective ~ {effective_sample_rate_hz:.1f} Hz')
plt.grid(True)
plt.show()

# Clean up
try:
    ser.close()
    print('Serial port closed.')
except Exception as e:
    print('Warning: could not close serial port:', e)